<a href="https://colab.research.google.com/github/thabir303/CSE-837-ML/blob/main/hw5/logistic-regression-lab-exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚯 Lecture 11 Lab: Logistic regression and spam detection

<img src="https://github.com/joshuagrossman/mse125-labs-public/blob/main/hw5/img/spam-email.png?raw=1" alt= “spam-email” width="500" />

## ✅ Setup and data import
In this lab, we will work with a [classic dataset](https://archive.ics.uci.edu/dataset/94/spambase) of 4,601 emails classified as spam or not spam.

In [13]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option("display.float_format", lambda v: f"{v:0.3f}")
sns.set_theme(style="whitegrid", context="talk")

spam = pd.read_csv("https://jdgrossman.com/assets/spam.csv")
spam.sample(10, random_state=42)

,make,address,all,3d,our,over,remove,internet,order,mail,...,char_semicolon,char_left_paren,char_left_bracket,char_exclamation,char_dollar,char_pound,capital_run_length_average,capital_run_length_longest,capital_run_length_total,is_spam
3683,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,1.000,1,3,0
4412,0.710,0.000,0.710,0.000,0.000,0.000,0.000,0.000,0.000,0.710,...,0.000,0.000,0.000,0.000,0.000,0.000,1.032,2,32,0
2584,0.000,0.000,0.910,0.000,0.000,0.000,0.000,0.450,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,1.320,7,103,0
69,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.201,0.000,0.000,0.100,0.000,4.548,59,141,1
1844,0.000,0.000,0.540,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.188,0.047,0.000,0.000,0.000,1.745,12,89,0
33,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.302,0.000,1.700,5,17,1
3085,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.314,0.000,0.000,2.473,10,47,0
4034,0.510,0.000,0.510,0.000,1.530,0.510,0.000,0.000,0.000,0.000,...,0.000,0.270,0.000,0.000,0.000,0.000,1.983,24,121,0
4470,0.000,0.000,0.440,0.000,0.000,0.440,0.000,0.000,0.000,0.000,...,0.000,0.074,0.000,0.149,0.000,0.000,1.115,2,29,0
4579,0.270,0.050,0.100,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.607,0.064,0.036,0.055,0.000,0.202,3.766,43,1789,0


## ♨️ Warm up

How many emails are in the database?

What fraction of the emails in the database are spam?

Which email contains the highest percentage of words matching "money"? What percentage of words in that email match "money"?

In [14]:
num_emails = len(spam)
spam_fraction = spam['is_spam'].mean()
money_idx = spam['money'].idxmax()
money_row = spam.loc[money_idx, ['money']]
print(f"Total emails: {num_emails}")
print(f"Fraction spam: {spam_fraction:.3f}")
print(f"Email with highest 'money' share: row {money_idx} with {money_row['money']:.2f}% of words")
spam.loc[money_idx]

Total emails: 4601
Fraction spam: 0.394
Email with highest 'money' share: row 545 with 12.50% of words


make                          0.000
address                       0.000
all                           0.000
3d                            0.000
our                           0.000
over                          0.000
remove                        0.000
internet                      0.000
order                         0.000
mail                          0.000
receive                       0.000
will                          0.000
people                        0.000
report                        0.000
addresses                     0.000
free                          0.000
business                      0.000
email                         0.000
you                          12.500
credit                        0.000
your                          0.000
font                          0.000
000                           0.000
money                        12.500
hp                            0.000
hpl                           0.000
george                        0.000
650                         

## 🎲 Linear probability models (LPMs)
Using `statsmodels` we fit `is_spam` on an intercept plus `char_dollar`, `credit`, `money`, and `re`.
The intercept (0.335) is the baseline spam probability when all predictors are zero, and the `char_dollar` slope (≈0.586) means a one-point increase in the dollar-character share of an email raises its predicted spam probability by about 0.59 (59 percentage points).

In [15]:
import statsmodels.api as sm
from statsmodels.tools import add_constant

predictors = ['char_dollar', 'credit', 'money', 're']
X = add_constant(spam[predictors])
y = spam['is_spam']
lpm = sm.OLS(y, X).fit()
print(lpm.summary())

                            OLS Regression Results                            
Dep. Variable:                is_spam   R-squared:                       0.179
Model:                            OLS   Adj. R-squared:                  0.178
Method:                 Least Squares   F-statistic:                     250.3
Date:                Tue, 18 Nov 2025   Prob (F-statistic):          8.51e-195
Time:                        11:22:18   Log-Likelihood:                -2780.3
No. Observations:                4601   AIC:                             5571.
Df Residuals:                    4596   BIC:                             5603.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           0.3346      0.007     45.696      

The LPM’s fitted probabilities range from −0.81 to 3.85, so it clearly produces invalid values below 0 and above 1—classic evidence that a linear model isn’t well suited for probabilities.

In [16]:
pred_probs = lpm.predict(X)
min_prob = pred_probs.min()
max_prob = pred_probs.max()
print(f"Smallest predicted probability: {min_prob:.3f}")
print(f"Largest predicted probability: {max_prob:.3f}")
print("Any predictions < 0?", (pred_probs < 0).any())
print("Any predictions > 1?", (pred_probs > 1).any())
pred_probs.describe()

Smallest predicted probability: -0.813
Largest predicted probability: 3.849
Any predictions < 0? True
Any predictions > 1? True


count   4601.000
mean       0.394
std        0.207
min       -0.813
25%        0.335
50%        0.335
75%        0.395
max        3.849
dtype: float64

## 🎰 Odds functions
A quick probability↔odds toolkit helps diagnose logistic models and reason about multiplicative effects.

Write two functions:
- A function to convert probabilities to odds.
- A function to convert odds to probabilities

Test your functions by making sure that 2:1 odds returns a 2/3 probability, and vice versa.

Finally, suppose my probability of winning is 60%. If I double my odds of winning, what is my new probability of winning?

In [17]:
def prob_to_odds(p):
    p = np.asarray(p)
    return p / (1 - p)

def odds_to_prob(o):
    o = np.asarray(o)
    return o / (1 + o)

print("Prob 2/3 -> odds:", prob_to_odds(2/3))
print("Odds 2 -> prob:", odds_to_prob(2))

p_win = 0.60
odds_win = prob_to_odds(p_win)
doubled_odds_prob = odds_to_prob(2 * odds_win)
print(f"Original prob 60% -> odds {odds_win:.3f}; doubling odds gives prob {doubled_odds_prob:.3f}")

Prob 2/3 -> odds: 1.9999999999999998
Odds 2 -> prob: 0.6666666666666666
Original prob 60% -> odds 1.500; doubling odds gives prob 0.750


## 🪙 Fitting a logistic regression model

We can fit a logistic regression model with the same covariates as above with the following code:

In [18]:
import statsmodels.api as sm
from statsmodels.tools import add_constant

logit_predictors = ['char_dollar', 'credit', 'money', 're']
X_logit = add_constant(spam[logit_predictors])
y_logit = spam['is_spam']

logit_model = sm.GLM(y_logit, X_logit, family=sm.families.Binomial())
logit_results = logit_model.fit()
logit_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                is_spam   No. Observations:                 4601
Model:                            GLM   Df Residuals:                     4596
Model Family:                Binomial   Df Model:                            4
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -2213.9
Date:                Tue, 18 Nov 2025   Deviance:                       4427.8
Time:                        11:22:18   Pearson chi2:                 1.00e+10
No. Iterations:                     8   Pseudo R-squ. (CS):             0.3152
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
const          -1.0666      0.043    -24.680      0.000      -1.151      -0.982
char_dollar    11.8176      0.605     19.549      0.000      10.633      13.002
credit          2.3119      0.343      6.741      0.000       1.640       2.984
money           1.9933      0.248      8.022      0.000       1.506       2.480
re             -0.7755      0.099     -7.805      0.000      -0.970      -0.581
===============================================================================
"""

Interpret the fitted coefficients like this:

1. **Intercept (`const`)**
   - **Log-odds:** The intercept is −1.07, meaning that when an email has 0% dollar characters, 0% credit mentions, 0% "money" words, and 0% "re" words, the log-odds of spam equals −1.07.
   - **Odds:** Exponentiating gives odds of exp(−1.07) ≈ 0.34, so in that baseline scenario we expect roughly 1 spam for every 3 non-spam emails.
   - **Probability:** Converting those odds back to a probability yields about 0.26 (26%) chance the email is spam when all predictors are zero.

2. **`money` coefficient**
   - **Log-odds:** The slope is 1.99, so each 1 percentage point increase in the share of words that match "money" adds 1.99 to the log-odds of spam.
   - **Odds:** On the odds scale this is a multiplier of exp(1.99) ≈ 7.34, meaning the spam odds jump more than sevenfold for each additional point of "money" share, holding other features fixed.
   - **Probability:** Starting from the baseline 26% probability, adding one percentage point of "money" raises the predicted probability to about 0.72 (a ≈46 percentage-point gain). The divide-by-4 trick gives a similar ≈0.50 point approximation, so both views agree that "money" is a very strong signal of spam.

In [19]:
intercept = logit_results.params['const']
money_coef = logit_results.params['money']

intercept_odds = np.exp(intercept)
money_odds_ratio = np.exp(money_coef)

intercept_prob = odds_to_prob(intercept_odds)
prob_with_money_plus1 = odds_to_prob(intercept_odds * money_odds_ratio)
prob_change = prob_with_money_plus1 - intercept_prob

print(f"Intercept (log-odds): {intercept:.3f}")
print(f"Intercept odds: {intercept_odds:.3f}")
print(f"Baseline spam probability when predictors are zero: {intercept_prob:.3f}")

print(f"'money' coefficient (log-odds per 1% point): {money_coef:.3f}")
print(f"'money' odds ratio (multiplicative effect): {money_odds_ratio:.3f}")
print(f"Approximate probability change for +1 point in 'money': {prob_change:.3f}")
print(f"Divide-by-4 heuristic (probability points): {money_coef / 4:.3f}")

Intercept (log-odds): -1.067
Intercept odds: 0.344
Baseline spam probability when predictors are zero: 0.256
'money' coefficient (log-odds per 1% point): 1.993
'money' odds ratio (multiplicative effect): 7.340
Approximate probability change for +1 point in 'money': 0.460
Divide-by-4 heuristic (probability points): 0.498
